In [265]:
import pandas as pan
import sklearn
from sklearn import metrics
from sklearn.cluster import KMeans, Birch
from sklearn.metrics import silhouette_score

datafile = "Redacted-Department _ CAD Reconciliation.csv"
data = pan.read_csv(datafile)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   XREF ID                2200 non-null   int64 
 1   DISPATCH UNIT          2200 non-null   object
 2   DISPATCH CREATED DATE  2200 non-null   object
 3   INCIDENT NUMBER        2200 non-null   object
 4   1ST UNIT ON SCENE      1772 non-null   object
 5   ALARM DATE TIME        2169 non-null   object
 6   CALL COMPLETE          2169 non-null   object
 7   SHIFT                  2131 non-null   object
dtypes: int64(1), object(7)
memory usage: 137.6+ KB


Our range is 2200 entries

In [266]:
print("Missing data per column (%)")
data.isna().mean() * 100

Missing data per column (%)


XREF ID                   0.000000
DISPATCH UNIT             0.000000
DISPATCH CREATED DATE     0.000000
INCIDENT NUMBER           0.000000
1ST UNIT ON SCENE        19.454545
ALARM DATE TIME           1.409091
CALL COMPLETE             1.409091
SHIFT                     3.136364
dtype: float64

There are some inconsistencies in the incident number formatting, with some not containing dashes. There are also multiple instances of the dispatch unit having multiple values separated by commas and will occasionally be surrounded by quotation marks. This will effect our analysis of the who is resolving alarms.

We'll drop data that is incomplete to parse data without interruptions by missing information.

In [267]:
data = data.dropna()
data['ALARM DATE TIME'] = pan.to_datetime(data['ALARM DATE TIME'])
data['CALL COMPLETE'] = pan.to_datetime(data['CALL COMPLETE'])

data['TIME DIFFERENCE'] = data['CALL COMPLETE'] - data['ALARM DATE TIME']
data['TIME DIFFERENCE'].mean()

/tmp/ipykernel_315163/3891229893.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['ALARM DATE TIME'] = pan.to_datetime(data['ALARM DATE TIME'])
/tmp/ipykernel_315163/3891229893.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['CALL COMPLETE'] = pan.to_datetime(data['CALL COMPLETE'])


Timedelta('87 days 07:22:03.593023256')

The average call completion time is 87 days, 7 hours, and 22 minutes.

In [268]:
data['DISPATCH UNIT'] = data['DISPATCH UNIT'].str.replace(r"^\w\s,", "")
data['COUNT UNITS'] = data['DISPATCH UNIT'].str.count(',') + 1
data['COUNT UNITS'].mean()

np.float64(1.4779069767441861)

There is an average of between 1 and 2 units being dispatched at any time, with the literal value being 1.478 units on average.

In [269]:
data['SHIFT'].mode()

0    A
Name: SHIFT, dtype: object

The shift that is dispatched most often is shift A.

In [270]:
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
data['DAYS OF WEEK'] = pan.Categorical(data['ALARM DATE TIME'].dt.day_name(), categories=days, ordered=True)
data['HOURS'] = data['ALARM DATE TIME'].dt.hour

matrix = data.pivot_table(
    index='HOURS',
    columns = 'DAYS OF WEEK',
    values='XREF ID',
    aggfunc='count',
    fill_value=0,
    observed=False,
    margins=True,
    margins_name="Totals"
)

print(matrix)


DAYS OF WEEK  Sunday  Monday  Tuesday  Wednesday  Thursday  Friday  Saturday  \
HOURS                                                                          
0                  4       7        6          4         1       2         7   
1                  7       8        7          7         4       3         5   
2                  7       4        3          2         2       3         7   
3                  4       6        8          9         1      10         4   
4                  4       2        4          5         2       6         4   
5                  4       8        3          6         5       3         2   
6                  9       5        5          6         6       8         6   
7                  7      13       13          6         7      11         4   
8                 11      11        9         13         7       6         4   
9                 11       8       14         11        12      12         7   
10                11      16       12   

In [ ]:
matrix = data.pivot_table(
    index='HOURS',
    columns = 'DAYS OF WEEK',
    values='XREF ID',
    aggfunc='count',
    fill_value=0,
    observed=False,
)

X = matrix.values
kmeans = KMeans(n_clusters=3, random_state=10,)
kmeans_titles = kmeans.fit_predict(X)

birch = Birch(threshold=0.01, n_clusters =3)
birch_titles = birch.fit_predict(X)

kmeans_score = silhouette_score(X, kmeans_titles)
birch_score = silhouette_score(X, birch_titles)

print(kmeans_score)
print(birch_score)

0.326139922806979
0.26093195747580833


The Kmeans method performs better at the given parameters. 